In [ ]:
import tensorflow as tf
import numpy as np
import math
import os
import pprint

from model import Model

In [ ]:
DST_INPUT_SIZE = 128
NUM_CLASS = 21

In [ ]:
def _parse_function(exampleProto):
    features={
            'image': tf.FixedLenFeature((), tf.string, default_value=""),
            'label': tf.FixedLenFeature((), tf.int64, default_value=0),
            'path': tf.FixedLenFeature((), tf.string, default_value="")
        }
    parsedFeatures = tf.parse_single_example(exampleProto, features)  # データ構造を解析
     
    return parsedFeatures["image"], parsedFeatures["label"], parsedFeatures["path"]
 
# データ解析（２）
def read_image(images, labels, paths):     
    image = tf.decode_raw(images, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = image / 255  # 画像データを、0～1の範囲に変換する
    image = tf.reshape(image, [DST_INPUT_SIZE, DST_INPUT_SIZE, 3])
    
    label = tf.cast(labels, tf.int32)
    label = tf.one_hot(label, NUM_CLASS)
    
    pathCodePoints = tf.strings.unicode_decode(input=paths, input_encoding='UTF-8')
    path = tf.strings.unicode_encode(pathCodePoints, output_encoding='UTF-8')
     
    return image, label, path

In [ ]:
names = {
    "iguchi": 0,
    "ushio": 1,
    "kakizaki": 2,
    "kageyama": 3,
    "kato": 4,
    "saito": 5,
    "sasaki_k": 6,
    "sasaki_m": 7,
    "takase": 8 ,
    "takamoto":9 ,
    "higashimura": 10,
    "kanemura": 11,
    "kawata": 12,
    "kosaka": 13,
    "tomita": 14,
    "nibu": 15,
    "hamagishi": 16,
    "matsuda": 17,
    "miyata": 18,
    "watanabe": 19,
    "kamimura": 20
}

In [ ]:
ckptPath= 'log/data_YYYY-MM-DD_hhmm/model.ckpt-NNNN'

In [ ]:
config = tf.ConfigProto(
    device_count = {'GPU': 0}
)

testDataNum = sum([1 for record  in tf.python_io.tf_record_iterator("test_tf_file_128x128.tfrecords")])
batchSize = 50

files = []
accuracyResults = []
actualResults= []
expectResults = []

with tf.Graph().as_default():
        
    # 入力層
    filenames = tf.placeholder(tf.string, shape=[None])
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(_parse_function, batchSize)  # レコードを解析し、テンソルに変換
    dataset = dataset.map(read_image, batchSize)  # データの形式、形状を変更
    dataset = dataset.batch(batchSize)  # 連続するレコードをバッチに結合
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)  # イテレータを作成
    images, labels, paths = iterator.get_next()  # イテレータの次の要素を取得
    
    initOp = iterator.make_initializer(dataset)  # イテレータを初期化
        
    model = Model(images, DST_INPUT_SIZE, NUM_CLASS, batchSize, trainable=False, labels=labels)
        
    with tf.Session(config=config) as session:
        session.run(initOp, feed_dict={filenames: ["test_tf_file_128x128.tfrecords"]})  # データの初期化

        saver = tf.train.Saver()
        
        saver.restore(session, ckptPath)

        for i in range(math.ceil(testDataNum / batchSize)):
            file, accuracyResult, actualResult, expectResult = session.run([paths, model.accuracy, model.logits, labels], feed_dict={model.keepProb: 1.0, model.isTraining: False})
            files.extend(file)
            accuracyResults.append(accuracyResult)
            actualResults.extend(actualResult)
            expectResults.extend(expectResult)
            
goods = []
bads = []
for index, (actual, expect) in enumerate(zip(actualResults, expectResults)):
    if np.argmax(actual) == np.argmax(expect):
        goods.append(files[index])
    else:
        bads.append(files[index])
    
    compareResult = []
    for expect, actual in zip(expectResults, actualResults):
        compareResult.append((np.argmax(expect), np.argmax(actual)))
        
print('accuracy: ', sum(accuracyResults) / len(accuracyResults))

In [ ]:
accuracyResultAndName = []

for index in range(len(names)):
    memberName = [key for key, value in names.items() if value == index][0]
    allTestFile = [filename.decode('utf-8') for filename in files if memberName in filename.decode('utf-8')]
    gooddResults = [filename.decode('utf-8') for filename in goods if memberName in filename.decode('utf-8')]
    accuracyResultAndName.append({'name': memberName, 'accuracy': len(gooddResults) / len(allTestFile)})

In [ ]:
accuracyResultAndName

In [ ]:
tfrecordFile = "train_tf_file_128x128.tfrecords"
trainDataNum = sum([1 for record  in tf.python_io.tf_record_iterator(tfrecordFile)])
batchSize = 50

config = tf.ConfigProto(
    device_count = {'GPU': 0}
)

files = []
accuracyResults = []
actualResults= []
expectResults = []

with tf.Graph().as_default():
        
    # 入力層
    filenames = tf.placeholder(tf.string, shape=[None])
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(_parse_function, batchSize)  # レコードを解析し、テンソルに変換
    dataset = dataset.map(read_image, batchSize)  # データの形式、形状を変更
    dataset = dataset.batch(batchSize)  # 連続するレコードをバッチに結合
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)  # イテレータを作成
    images, labels, paths = iterator.get_next()  # イテレータの次の要素を取得
    
    initOp = iterator.make_initializer(dataset)  # イテレータを初期化
        
    model = Model(images, labels, DST_INPUT_SIZE, NUM_CLASS, batchSize, trainable=False)
        
    with tf.Session(config=config) as session:
        session.run(initOp, feed_dict={filenames: [tfrecordFile]})  # データの初期化

        saver = tf.train.Saver()
        
        saver.restore(session, ckptPath)
        
        for i in range(int(testDataNum / batchSize)):
            file, accuracyResult, actualResult, expectResult = session.run([paths, model.accuracy, model.logits, labels], feed_dict={model.keepProb: 1.0, model.isTraining: False})
            files.extend(file)
            accuracyResults.append(accuracyResult)
            actualResults.extend(actualResult)
            expectResults.extend(expectResult)


goods = []
bads = []
for index, (actual, expect) in enumerate(zip(actualResults, expectResults)):
    if np.argmax(actual) == np.argmax(expect):
        goods.append(files[index])
    else:
        bads.append(files[index])
    
    compareResult = []
    for expect, actual in zip(expectResults, actualResults):
        compareResult.append((np.argmax(expect), np.argmax(actual)))
        
print('accuracy: ', sum(accuracyResults) / len(accuracyResults))